<a href="https://colab.research.google.com/github/armandoordonez/deeplearning/blob/main/CNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# esta celda
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

In [ ]:
# Cargar el conjunto de datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Preprocesamiento para MLP (aplanar imágenes y normalizar)
x_train_flat = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test_flat = x_test.reshape(x_test.shape[0], -1) / 255.0

In [ ]:
# Preprocesamiento para CNN (reformatear y normalizar)
x_train_cnn = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test_cnn = x_test.reshape(-1, 28, 28, 1) / 255.0

In [ ]:
# Definir y optimizar MLP con GridSearchCV
param_grid = {'hidden_layer_sizes': [(128,), (256,), (128, 64)],
              'activation': ['relu', 'tanh'],
              'alpha': [0.0001, 0.001],
              'learning_rate_init': [0.001, 0.01]}

In [ ]:
mlp = MLPClassifier(max_iter=10)
grid_search = GridSearchCV(mlp, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(x_train_flat[:10000], y_train[:10000])  # Usamos un subconjunto por tiempo de cómputo

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=3, estimator=MLPClassifier(max_iter=10), n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.001],
                         'hidden_layer_sizes': [(128,), (256,), (128, 64)],
                         'learning_rate_init': [0.001, 0.01]},
             verbose=1)

In [ ]:
# Evaluar MLP en el conjunto de prueba
best_mlp = grid_search.best_estimator_
y_pred_mlp = best_mlp.predict(x_test_flat)
mlp_acc = accuracy_score(y_test, y_pred_mlp)
print(f"Mejor MLP Accuracy: {mlp_acc:.4f}")

Mejor MLP Accuracy: 0.9598


In [ ]:
# Definir una CNN simple
model_cnn = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compilar y entrenar la CNN
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_cnn.fit(x_train_cnn, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 37s 21ms/step - accuracy: 0.9041 - loss: 0.3227 - val_accuracy: 0.9787 - val_loss: 0.0724
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9825 - loss: 0.0557 - val_accuracy: 0.9847 - val_loss: 0.0541
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 36s 21ms/step - accuracy: 0.9889 - loss: 0.0337 - val_accuracy: 0.9833 - val_loss: 0.0579
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.9936 - loss: 0.0198 - val_accuracy: 0.9870 - val_loss: 0.0491
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.9953 - loss: 0.0139 - val_accuracy: 0.9898 - val_loss: 0.0500


In [ ]:
# Evaluar CNN
cnn_loss, cnn_acc = model_cnn.evaluate(x_test_cnn, y_test, verbose=0)
print(f"CNN Accuracy: {cnn_acc:.4f}")

CNN Accuracy: 0.9873


In [ ]:
# Comparación final
diff = cnn_acc - mlp_acc
print(f"Diferencia de rendimiento CNN - MLP: {diff:.4f}")

Diferencia de rendimiento CNN - MLP: 0.0275


In [ ]:
import torch
import pickle

# Nombre del archivo donde se guardará el modelo
model_filename = "cnn_mnist.pkl"

# Guardar el modelo usando pickle
with open(model_filename, "wb") as f:
    pickle.dump(model_cnn, f)

print(f"Modelo guardado como {model_filename}")

# Si estás en Google Colab, permite la descarga
from google.colab import files
files.download(model_filename)

Modelo guardado como cnn_mnist.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>